In [1]:
from bokeh.layouts import gridplot, layout
from bokeh.models import  CustomJS
from bokeh.models.widgets import CheckboxGroup
from bokeh.plotting import figure, show, output_notebook
import numpy as np

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
file = '/vol01/active_projects/anthony/fhd_p4_eeg_pheno_v4_beta.csv'
arr = np.loadtxt(open(file, 'rb'),delimiter=",")

chans = ['FP1-F3', 'FP2-F4', 'FP1-F7', 'FP2-F8', 'F7-F3',
         'F8-F4', 'F7-T7', 'F8-T8', 'F3-C3', 'F4-C4', 'FZ-CZ',
         'CZ-PZ', 'T7-C3', 'T8-C4', 'T7-P7', 'T8-P8', 'C3-P3', 'C4-P4',
         'P7-P3', 'P8-P4', 'P7-O1', 'P8-O2', 'P3-O1', 'P4-O2', 'PZ-O1',
         'PZ-O2', 'O1-O2', 'CZ-C3', 'CZ-C4', 'PZ-P3', 'PZ-P4', 'F7-C3',
         'F8-C4', 'FP1-FP2', 'F3-FZ', 'FZ-F4']

# set up ages on x axis at intervals of .2 from 12-31 
xaxis = []
count = 11.8
for i in range(96):
    count += float(.2)
    xaxis.append(round(count,2))
    
xaxis_arr = np.array(xaxis)

In [4]:
na = arr.reshape(7,36,6,96)

In [5]:
figlist = [figure(title= chans[i],plot_width=200,plot_height=200) for i in range(36)]

linelist=[]

#output_file('bokeh_js.html')
count = 0
for fig in figlist:
    
    fig.x_range = figlist[0].x_range
    fig.y_range = figlist[0].y_range
    
    fig.yaxis.axis_label = 'amplitude'
    fig.xaxis.axis_label = 'age'
    
    # first dimension is frequency, 2nd = channel name 
    line0 = fig.line(xaxis_arr, na[0][count][2], line_width=2, color='#1f77b4')
    line1 = fig.line(xaxis_arr, na[1][count][2], line_width=2, color='#ff7f0e')
    line2 = fig.line(xaxis_arr, na[2][count][2], line_width=2, color='#2ca02c')
    line3 = fig.line(xaxis_arr, na[3][count][2], line_width=2, color='#d62728')
    line4 = fig.line(xaxis_arr, na[4][count][2], line_width=2, color='#9467bd')
    line5 = fig.line(xaxis_arr, na[5][count][2], line_width=2, color='#8c564b')
    line6 = fig.line(xaxis_arr, na[6][count][2], line_width=2, color='#e377c2')
    count+=1
    
    linelist+=[[line0,line1,line2,line3,line4,line5,line6]]
    
# create checkbox group with labels 
checkbox = CheckboxGroup(labels=["low theta", "high theta", "low alpha", "high alpha", "low beta", "mid beta", "high beta"], 
                             active=[0,1,2,3,4,5,6])


# creates Glyph Renderer(id=......) for every line in each subplot...
lines = []
for figid, elem in enumerate(linelist):
    for lineid, line in enumerate(elem):
        itera = ('_'.join(['line',str(figid),str(lineid)]),line)
        lines.append(itera)
        
# use python to concat JS
checkbox_code = ''.join([elem[0]+'.visible=checkbox.active.includes('+elem[0].split('_')[-1]+');' for elem in lines])

In [6]:
# callback 
checkbox.callback = CustomJS(args={key:value for key,value in lines+[('checkbox',checkbox)]}, code=checkbox_code)


# arrange & plot
grid = gridplot(figlist, ncols=6)

l = layout([checkbox], [grid])
show(l)